<a href="https://colab.research.google.com/github/SonOf1998/ProblemSet4/blob/main/ps4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First install the package that makes it easy to get the required images.

In [1]:
!pip install openimages

     |████████████████████████████████| 133kB 5.3MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 71kB 5.4MB/s 
     |████████████████████████████████| 6.7MB 7.4MB/s 
     |████████████████████████████████| 296kB 54.3MB/s 
     |████████████████████████████████| 137.3MB 91kB/s 
  Created wheel for ImageHash: filename=ImageHash-4.1.0-py2.py3-none-any.whl size=291991 sha256=7db2fa771612a1d3545b742d4aa61d91778e9a3916eda5a267dc41236187250d
  Stored in directory: /root/.cache/pip/wheels/07/1c/dc/6831446f09feb8cc199ec73a0f2f0703253f6ae013a22f4be9
Successfully built ImageHash
ERROR: botocore 1.19.13 has requirement urllib3<1.26,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [2]:
from openimages.download import download_images
import os
import shutil

In [3]:
# removes every directory from the directory given in the parameter
def clear_workdir(workdir):
  for filename in os.listdir(workdir):
    filepath = os.path.join(workdir, filename)
    if os.path.isdir(filepath):
      shutil.rmtree(filepath)

# creates empty directories for training, validation and testing data
def make_set_directory(set_name, classes):
  os.mkdir(set_name)
  for cls in classes:
    os.mkdir(os.path.join(set_name, cls))

In [4]:
workdir = os.getcwd()
clear_workdir(workdir)

# These are the classes I selected for the exercise..
# For whatever reason download_images() fails if I don't
# use upper case for the initial letter of the class' strings
classes = ["Car", "Bus", "Train"]
download_images(workdir, classes, exclusions_path=None, limit=600)

# Converts class strings to lowercase letters
# as download_images() make dirs with only lowercase names
for i in range(len(classes)):
  classes[i] = classes[i].lower()

set_dirs = ["training", "validation", "testing"]
for set_dir in set_dirs:
  make_set_directory(set_dir, classes)

nb_training = 400
nb_validation = 100
nb_testing = 100

for cls in classes:
  path_to_class = os.path.join(cls, "images")
  for i, filename in enumerate(os.listdir(path_to_class)):
    full_path_to_pic = os.path.join(path_to_class, filename)
    if i < nb_training:
      shutil.move(full_path_to_pic, os.path.join(workdir, set_dirs[0], cls, filename))
    elif i < nb_training + nb_validation:
      shutil.move(full_path_to_pic, os.path.join(workdir, set_dirs[1], cls, filename))
    else:
      shutil.move(full_path_to_pic, os.path.join(workdir, set_dirs[2], cls, filename))
  
  # we moved every picture to our train/valid/test set
  # so we can delete the empty directory
  shutil.rmtree(os.path.join(workdir, cls))



2020-11-08  16:10:12 INFO NumExpr defaulting to 2 threads.
2020-11-08  16:10:15 INFO Downloading 600 train images for class 'car'
100%|██████████| 600/600 [00:13<00:00, 43.66it/s]
2020-11-08  16:10:29 INFO Downloading 600 train images for class 'bus'
100%|██████████| 600/600 [00:14<00:00, 41.70it/s]
2020-11-08  16:10:43 INFO Downloading 600 train images for class 'train'
100%|██████████| 600/600 [00:13<00:00, 43.20it/s]


In [5]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import ModelCheckpoint, History, EarlyStopping

In [6]:
img_height = 256     # Input image height
img_width  = 256     # Input image width
batch_size = 32
class_mode = 'categorical'
color_mode = 'rgb' 

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_iter = train_datagen.flow_from_directory(
    os.path.join(workdir, set_dirs[0]),
    target_size=(img_height, img_width),
    batch_size = batch_size,
    class_mode = class_mode,
    color_mode = color_mode
)

validation_iter = train_datagen.flow_from_directory(
    os.path.join(workdir, set_dirs[1]),
    target_size=(img_height, img_width),
    batch_size = batch_size,
    class_mode = class_mode,
    color_mode = color_mode
)

test_iter = test_datagen.flow_from_directory(
    os.path.join(workdir, set_dirs[2]),
    target_size=(img_height, img_width),
    batch_size = 1,
    class_mode = class_mode,
    color_mode = color_mode
)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [7]:
base_model = InceptionV3(input_shape=(img_height, img_width, 3),
                    weights="imagenet",
                    include_top=False,
                    classes=3)

inputs = Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
outputs = Dense(3, activation="softmax")(x)

model = Model(inputs, outputs)

checkpoint = ModelCheckpoint("chk.chk", save_weights_only=True, save_best_only=True, monitor="val_accuracy", verbose=1)

87916544/87910968 [==============================] - 1s 0us/step


In [8]:
base_model.trainable = False
model.compile(optimizer="rmsprop", loss="mse", metrics=["acc"])

model.fit(
    train_iter,
    epochs=30,
    validation_data=validation_iter,
    callbacks=[checkpoint]) 

Epoch 1/30
38/38 [==============================] - ETA: 0s - loss: 0.4356 - acc: 0.3333

2020-11-08  16:15:09 WARNING Can save best model only with val_accuracy available, skipping.


38/38 [==============================] - 232s 6s/step - loss: 0.4356 - acc: 0.3333 - val_loss: 0.4444 - val_acc: 0.3333
Epoch 2/30
28/38 [=====================>........] - ETA: 47s - loss: 0.4394 - acc: 0.3409

KeyboardInterrupt: ignored

In [ ]:
base_model.trainable = True
model.compile(optimizer="rmsprop", loss="mse", metrics=["acc"])

model.fit(
    train_iter,
    epochs=30,
    validation_data=validation_iter,
    callbacks=[checkpoint]) 

In [ ]:
loss, acc = model.evaluate(test_iter)
print(acc)